In [1]:
import json
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os
from glob import glob

In [ ]:
def create_pascal_voc_xml(image_info, annotations, categories, output_dir):
    root = ET.Element("annotation")
    
    ET.SubElement(root, "folder").text = "VOC2012"
    ET.SubElement(root, "filename").text = image_info['file_name']
    
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(image_info['width'])
    ET.SubElement(size, "height").text = str(image_info['height'])
    ET.SubElement(size, "depth").text = "3"
    
    ET.SubElement(root, "segmented").text = "0"
    
    for ann in annotations:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = categories[ann['category_id'] - 1]['name']
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"
        
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(int(ann['bbox'][0]))
        ET.SubElement(bbox, "ymin").text = str(int(ann['bbox'][1]))
        ET.SubElement(bbox, "xmax").text = str(int(ann['bbox'][0] + ann['bbox'][2]))
        ET.SubElement(bbox, "ymax").text = str(int(ann['bbox'][1] + ann['bbox'][3]))
    
    xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="    ")
    
    with open(os.path.join(output_dir, os.path.splitext(image_info['file_name'])[0] + ".xml"), "w") as f:
        f.write(xml_str)


In [ ]:
def convert_coco_to_pascal(coco_json, output_dir):
    with open(coco_json, 'r') as f:
        coco_data = json.load(f)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_dict = {img['id']: img for img in coco_data['images']}
    
    for img_id, img_info in image_dict.items():
        annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_id]
        create_pascal_voc_xml(img_info, annotations, coco_data['categories'], output_dir)
    
    print(f"Conversion complete. Pascal VOC XML files saved in {output_dir}")

In [ ]:
def get_annotation_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    filename = root.find('filename').text
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    objects = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        difficult = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        
        objects.append({
            'name': name,
            'difficult': difficult,
            'bbox': [xmin, ymin, xmax - xmin, ymax - ymin]
        })
    
    return {
        'filename': filename,
        'width': width,
        'height': height,
        'objects': objects
    }

In [ ]:
def convert_pascal_to_coco(pascal_dir, output_json):
    xml_files = sorted(glob(os.path.join(pascal_dir, '*.xml')))
    
    coco_data = {
        'images': [],
        'annotations': [],
        'categories': []
    }
    
    categories = {}
    annotation_id = 1
    
    for img_id, xml_file in enumerate(xml_files, 1):
        ann = get_annotation_from_xml(xml_file)
        
        # Add 'train/' prefix to file_name if it's not already there
        file_name = ann['filename']
        if not file_name.startswith('train/'):
            file_name = 'train/' + file_name
        
        coco_data['images'].append({
            'id': img_id,
            'file_name': file_name,
            'width': ann['width'],
            'height': ann['height']
        })
        
        for obj in ann['objects']:
            if obj['name'] not in categories:
                categories[obj['name']] = len(categories) + 1
                coco_data['categories'].append({
                    'id': categories[obj['name']],
                    'name': obj['name'],
                    'supercategory': 'none'
                })
            
            coco_data['annotations'].append({
                'id': annotation_id,
                'image_id': img_id,
                'category_id': categories[obj['name']],
                'bbox': obj['bbox'],
                'area': obj['bbox'][2] * obj['bbox'][3],
                'iscrowd': 0,
                'difficult': obj['difficult']
            })
            annotation_id += 1
    
    with open(output_json, 'w') as f:
        json.dump(coco_data, f, indent=2)
    
    print(f"Conversion complete. COCO JSON file saved as {output_json}")
    print(f"Processed {len(xml_files)} XML files in sorted order.")
    print(f"Added 'train/' prefix to file names where necessary.")

In [ ]:
# 사용 예시
coco_json_path = '/data/ephemeral/home/dataset/train.json'
output_directory = '/data/ephemeral/home/dataset/pascal_voc_xml'
convert_coco_to_pascal(coco_json_path, output_directory)

In [ ]:
# 사용 예시
pascal_voc_dir = '/data/ephemeral/home/dataset/pascal_voc_xml'
output_coco_json = '/data/ephemeral/home/dataset/coco_from_pascal.json'
convert_pascal_to_coco(pascal_voc_dir, output_coco_json)